# **Imports**

In [ ]:
import os

from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.document_loaders import PyPDFLoader
from typing import List, Optional
from pydantic import BaseModel

load_dotenv()


True

## **Example of Ai Agent**

In [5]:
from langchain.agents import Tool, tool
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# --- Tool definitions ---
def sum_numbers(input: str) -> str:
    numbers = list(map(int, input.split()))
    return str(sum(numbers))

def multiply_numbers(input: str) -> str:
    numbers = list(map(int, input.split()))
    result = 1
    for num in numbers:
        result *= num
    return str(result)

tools = [
    Tool(
        name="AdditionTool",
        func=sum_numbers,
        description="Use this to add numbers. Input should be space-separated numbers like '2 3 5'."
    ),
    Tool(
        name="MultiplicationTool",
        func=multiply_numbers,
        description="Use this to multiply numbers. Input should be space-separated numbers like '2 3 5'."
    ),
]

# --- Prompt Template ---
prompt = hub.pull("hwchase17/openai-functions-agent")

# --- LLM and Agent Setup ---
llm = ChatOpenAI(model="gpt-4.1-nano")
agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- Run the agent ---
result = executor.invoke({"input": "Please add 10 and 5"})
print("Result:", result["output"])

result2 = executor.invoke({"input": "Now multiply 4 and 3 and 2"})
print("Result:", result2["output"])




> Entering new AgentExecutor chain...

Invoking: `AdditionTool` with `10 5`


15The sum of 10 and 5 is 15.

> Finished chain.
Result: The sum of 10 and 5 is 15.


> Entering new AgentExecutor chain...

Invoking: `MultiplicationTool` with `4 3 2`


24The result of multiplying 4, 3, and 2 is 24.

> Finished chain.
Result: The result of multiplying 4, 3, and 2 is 24.


In [ ]:
import os

from dotenv import load_dotenv
from langchain.output_parsers import PydanticOutputParser
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import PromptTemplate
from typing import Literal

load_dotenv()




class RouterAgentOutput(BaseModel):
    next_step: Literal[
        "hospital_agent", "movie_agent", "conversation", "document_retrieval"
    ] = Field(
        description="Determines which agent should handle the request next, if the user ask for a specific document related question use the: document_retrieval",
        examples=[
            "hospital_agent",
            "movie_agent",
            "conversation",
            "document_retrieval",
        ],
    )

ROUTER_TEMPLATE = """You are a router agent responsible for directing user queries to the appropriate service. 
Given a user request, classify it
QUERY: {query}

FORMAT_INSTRUCTIONS: {format_instructions}"""



OUTPUT_PARSER = PydanticOutputParser(pydantic_object=RouterAgentOutput)
PROMPT_FORMAT = PromptTemplate(
    template=ROUTER_TEMPLATE,
    input_variables=["query"],
    partial_variables={"format_instructions": OUTPUT_PARSER.get_format_instructions()},
)


class RouterAgent:
    def __init__(
        self, model: str = "gemini-2.0-flash-lite", temperature: float = 0.5, open=False
    ):
        llm = ChatVertexAI(model=model, temperature=temperature)
        self.chain = PROMPT_FORMAT | llm | OUTPUT_PARSER

    def invoke(self, query: str):
        return self.chain.invoke({"query": query})
